# M3 - Actividad
Carlos Moisés Chávez Jiménez A01637322
28/11/2021

## Descripción del código
Sistema multiagente necesario para simular una intersección controlada por señales de semáforos inteligentes.

### Verificamos nuestra versión de Python

In [78]:
import sys
sys.version

'3.9.5 (tags/v3.9.5:0a7dcbd, May  3 2021, 17:27:52) [MSC v.1928 64 bit (AMD64)]'

### Instalamos la librería AgentPy

In [79]:
!pip install agentpy

### Importamos las librerías a utilizar

In [80]:
# Model design
import agentpy as ap

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
import math
import random

### Declaramos la clase que contendrá los agentes a utilizar

- Semaforos: Controlarán el paso de los carros, y contarán con tres estados relacionados a sus tres colores. 
- Automoviles: Estos estarán guiados por los semaforos y se generarán con base en los parametros que se recibe.

In [81]:
class Interseccion(ap.Model):
  
  def setup(self):
    #Posiciones de los agentes
    semaforosPosicion = [(12,10),(8,10),(10,8),(10,12)]
    autoPosicion=[11,9,9,11]

    # Creamos los agentes
    cont=0
    self.semaforos = ap.AgentList(self, 4)
    # Designaremos los atributos de los semaforos
    self.semaforos.color = 1
    self.semaforos.direccion = 10000
    self.semaforos.cooldown = 0
    # Le asignaremos id a cada semaforo
    for semaforo in self.semaforos:
      semaforo.id=cont
      cont+=1

    # Creamos y definimos los atributos de los autos.
    self.autos = ap.AgentList(self, self.p.autos)
    self.autos.color = 3
    self.autos.id = 1000
    # Crearemos una bandera que nos indicará si el carro está en movimiento o detenido.
    self.autos.move = 1

    # Creamos el tablero con la intersección
    self.interseccion = ap.Grid(self, (20,20),torus=True, track_empty=True)
    self.interseccion.add_agents(self.semaforos,semaforosPosicion, empty = False)
    self.interseccion.add_agents(self.autos, random = True, empty = True)

    # Crearemos los automoviles de manera dinamica
    # Para manejar la dirección de cada automovil, lo haremos con 4 números
    # 0 - Arriba
    # 1 - Abajo
    # 2 - Derecha
    # 3 - Izquierda
    for auto in self.autos:
      # Asignaremos la dirección de manera random.
      direccion = random.randint(0,3)
      #Dependiendo de la dirección que tome el auto, sera el comportamiento de la instrucción move_to
      if direccion == 0:
        auto.direccion=direccion
        self.interseccion.move_to(auto,(random.randint(13,20),autoPosicion[3]))
      elif direccion == 1:
        auto.direccion=direccion
        self.interseccion.move_to(auto,(random.randint(0,7),autoPosicion[2]))
      elif direccion == 2:
        auto.direccion=direccion
        self.interseccion.move_to(auto,(autoPosicion[0],random.randint(0,7)))
      elif direccion == 3:
        auto.direccion=direccion
        self.interseccion.move_to(auto,(autoPosicion[1],random.randint(13,20)))


  def step(self):
    # Manejaremos el comportamiento de cada step
    semaforosID = 0
    valMin = 100000
    # Trataremos de buscar el semaforo que tenga al automovil más cerano
    for semaforo in self.semaforos:
      # Dentro de cada semaforo, buscaremos a los vecinos
      for vecino in self.interseccion.neighbors(semaforo,2):
        pos1 = self.interseccion.positions[vecino]
        pos2 = self.interseccion.positions[semaforo]
        distancia = math.sqrt( (pos1[0]-pos2[0])**2 + (pos1[1]-pos2[1])**2 )
        if(valMin>distancia and vecino.direccion==semaforo.id):
          valMin = distancia
          semaforosID = semaforo.id
    # Despues de haber buscado vecinos, si no encontramos ninguno, pondremos todos los semaforos en color amarillo.
    if valMin == 100000:
      self.semaforos.color = 1
    # Si encontramos un vecino, le pondremos luz verde
    else:
      semaforo = self.semaforos.select(self.semaforos.id == semaforosID)
      self.semaforos.color = 0
      # Cambiaremos las demás luces a rojo.
      # Para lo anterior, debemos identificar el cruce que tiene la luz verde.
      if semaforo[0].id == 0 or semaforo[0].id == 1:
        semaforo = self.semaforos.select(self.semaforos.id == 0)
        semaforo.color = 2
        semaforo = self.semaforos.select(self.semaforos.id == 1)
        semaforo.color = 2
      # El cruce opuesto al verde, se pondrá de rojo.
      else:
        semaforo = self.semaforos.select(self.semaforos.id == 2)
        semaforo.color = 2
        semaforo = self.semaforos.select(self.semaforos.id == 3)
        semaforo.color = 2

    # Definiremos que los autos se detengan con la luz roja.
    # Para esto, unicamente tomaremos los autos que se están moviendo.
    for auto in self.autos.select(self.autos.move==1):
      # Buscaremos los autos de las cuatro direcciones de movimiento
      # Autos que se mueven hacia arriba.
      if auto.direccion == 0:
        for vecino in self.interseccion.neighbors(auto,2):
          if vecino.color==0 and vecino.id==auto.direccion:
            auto.move=1
            break
        else:
          for vecino in self.interseccion.neighbors(auto):
            if vecino.color==3 and (self.interseccion.positions[vecino][0]-self.interseccion.positions[auto][0])==-1 and (self.interseccion.positions[vecino][1]-self.interseccion.positions[auto][1])==0:
              break
          else:
            self.interseccion.move_by(auto,(-1,0))

      # Autos que se mueven hacia abajo.
      elif auto.direccion == 1:
        for vecino in self.interseccion.neighbors(auto,2):
          if vecino.color==0 and vecino.id==auto.direccion:
            auto.move=1
            break
        else:
          for vecino in self.interseccion.neighbors(auto):
            if vecino.color==3 and (self.interseccion.positions[vecino][0]-self.interseccion.positions[auto][0])==1 and (self.interseccion.positions[vecino][1]-self.interseccion.positions[auto][1])==0:
              break
          else:
            self.interseccion.move_by(auto,(1,0))

      # Autos que se mueven hacia la derecha.
      elif auto.direccion == 2:
        for vecino in self.interseccion.neighbors(auto,2):
          if vecino.color==0 and vecino.id==auto.direccion:
            auto.move=1
            break
        else:
          for vecino in self.interseccion.neighbors(auto):
            if vecino.color==3 and (self.interseccion.positions[vecino][1]-self.interseccion.positions[auto][1])==1 and (self.interseccion.positions[vecino][0]-self.interseccion.positions[auto][0])==0:
              break
          else:
            self.interseccion.move_by(auto,(0,1))

      # Autos que se mueven hacia la izquierda.
      elif auto.direccion == 3:
        for vecino in self.interseccion.neighbors(auto,2):
          if vecino.color==0 and vecino.id==auto.direccion:
            auto.move=1
            break
        else:
          for vecino in self.interseccion.neighbors(auto):
            if vecino.color==3 and (self.interseccion.positions[vecino][1]-self.interseccion.positions[auto][1])==-1 and (self.interseccion.positions[vecino][0]-self.interseccion.positions[auto][0])==0:
              break
          else:
            self.interseccion.move_by(auto,(0,-1))

### Definimos los Parámetros

In [82]:
parameters = {
    'autos': 12,
    'steps': 100, # Limite de pasos (tiempo)
}

### Visualizamos y Simulamos

In [83]:
# Creamos la animación con los parámetros (Titulo, colores, números, etc) necesarios.
def animation_plot(model, ax):
    attr_grid = model.interseccion.attr_grid('color')
    color_dict = {0:'#FF0000', 1:'#FFFF00', 2:'#00FF00', 3:'#005BFF', None:'#000000'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Intersección controlada por señales de semáforos inteligentes\n"
                f"Steps: {model.t}, Automoviles: {len(model.autos)}")
fig, ax = plt.subplots()
model = Interseccion(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))